ProgEcon: Project 1

Group: Nikolaj and Fanny

In [5]:
import numpy as np
import pandas as pd

# APIs
from fredapi import Fred
from dstapi import DstApi

# plotting
import matplotlib.pyplot as plt
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.rcParams.update({'axes.grid':True,'grid.color':'black','grid.alpha':'0.25','grid.linestyle':'--'})
plt.rcParams.update({'font.size': 14})

# 1. Aggregate inflation in Denmark

## 1.1 Basic


In [6]:
PRIS113 = DstApi('PRIS113') 
PRIS113.tablesummary(language='en')


Table PRIS113: Consumer price index by type and time
Last update: 2025-10-10T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,TYPE,1,INDEKS,Consumer price index,INDEKS,Consumer price index,False
1,Tid,549,1980M01,1980M01,2025M09,2025M09,True


## 1.2 Instantaneous
$$
\begin{align*}
\pi_t^{12,\alpha} &= \left( \prod_{k=0}^{11} (1 + \pi_{t-k})^{\kappa(k,\alpha)} \right) - 1, \\[6pt]
\kappa(k,\alpha) &= \frac{(T - k)^{\alpha}}{\sum (T - k)^{\alpha} \, }T
\end{align*}
$$

## 1.3 Core inflation

## 1.4 Disaggregated inflation in Denmark

# 2. International Comparison

## 2.1 CPI vs HICP

## 2.2 Across countries

# 3. Extension